
## **Intent Classification with Orq.ai**

**Prerequisites**

In this tutorial we will use Orq.ai and Hugging Face secret API keys. Before you get started: find [Orq.ai API key](https://docs.orq.ai/docs/administer/api-keys) and [Hugging Face API key](https://huggingface.co/docs/hub/en/security-tokens) first. In Google Colab You can [safely store your API secret keys](https://www.youtube.com/watch?v=LPa51KxqUAw) by clicking the Key icon on the left and setting up the variables

Those are the variables that you need to set:

*   ```YOUR_HUGGING_FACE_API```: your API here
*   ```YOUR_ORQ_API_KEY```: your API here


load the API keys:

In [ ]:
from google.colab import userdata

# Use your Hugging Face API token
YOUR_HUGGING_FACE_API = userdata.get("YOUR_HUGGING_FACE_API")
YOUR_ORQ_API = userdata.get("YOUR_ORQ_API_KEY")



**Step 1: Install Dependencies**  
Start by installing the required packages. These include the Orq SDK and additional libraries for handling datasets and managing environment variables.

In [ ]:
!pip install orq-ai-sdk datasets huggingface_hub

**Step 2: Initialize the Orq Client**  

The Orq client allows you to communicate with the Orq platform. Set it up using your API key, which can be stored as an environment variable (ORQ_API_KEY) or passed directly.

In [ ]:
import os

from orq_ai_sdk import Orq

client = Orq(
  api_key=YOUR_ORQ_API,
)

### **Hugging Face**

Before proceeding, sign up for a free Hugging Face account if you don't already have one. You'll need an API key to access their datasets library. Retrieve your API key here after signing up or logging in.

**Step 3: Load a Dataset**  
Use the Hugging Face datasets library to load the dataset for intent classification. Here, we use a public dataset that contains user queries labeled with intents.

In [ ]:
from huggingface_hub import login

login(token=YOUR_HUGGING_FACE_API)

In [ ]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("Bhuvaneshwari/intent_classification")
df = dataset["train"].to_pandas()

In [ ]:
# Select the top 50 rows
df = df.head(50)

# Display the DataFrame
print(df)

# Optional: Check the shape and columns
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

In [ ]:
# Extract unique labels from the "intent" column
unique_labels = df["intent"].unique()
unique_labels

### **Adding Generated Responses to DataFrame**

This code adds an empty column to the DataFrame for storing generated responses, iterates through each row to generate a response based on the `text_input`, and then saves the updated DataFrame.

#### **Intent classification prompt**

This deployment is designed to classify user inputs into specific intents to understand the purpose behind their requests. The model will identify the most appropriate intent from a predefined set, enabling precise and context-aware responses.

Intent recognition is particularly useful when setting up chatbots

This is the prompt in Orq.ai:

```plaintext
You are tasked with identifying the intent behind user inputs based on the following intents:
'PlayMusic', 'AddToPlaylist', 'RateBook', 'Greetings', 'SearchScreeningEvent', 'BookRestaurant', 'GetWeather', 'Book Meeting', 'SearchCreativeWork', 'Cancellation', 'Affirmation', 'excitment'.

Here are some examples:

Input: "Hey there, how are you doing?"
Intent: Greetings

Input: "Play the album Abbey Road by The Beatles."
Intent: PlayMusic

Input: "Add this song to my workout playlist."
Intent: AddToPlaylist

Input: "This book deserves a solid five-star rating."
Intent: RateBook

ONLY OUTPUT THE LABEL WITHOUT ''

here is the input that needs intent classification: {{text}}



**Step 4: Invoke Orq Deployment**  

Integrate the Orq intent classification model by invoking a deployed model for predictions. Iterate through the dataset and store the results in a new column called "output".

In [ ]:
# Placeholder for the deployment key
deployment_key = "intent_classification"

# Create an empty list to store outputs
outputs = []

# Iterate through the rows of the DataFrame
for index, row in df.iterrows():
    text_input = row["text"]  # Extract the text column

    # Invoke the model for each text input
    generation = client.deployments.invoke(
        key=deployment_key,
        context={
            "environments": []
        },
        inputs={
            "text": text_input
        },
        metadata={
            "custom-field-name": "custom-metadata-value"
        }
    )

    # Append the output to the list
    outputs.append(generation.choices[0].message.content)


In [ ]:
 # Add the output column to the DataFrame
df["output"] = outputs

**Step 5: Evaluate Model Performance**  

Use metrics such as accuracy, precision, recall, and F1-score to assess the quality of your model's predictions.

In [ ]:

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

true_labels = df["intent"]
predicted_labels = df["output"]

# Calculate performance metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Print the metrics
print("Performance Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

**Next Steps**  

Congratulations! You've successfully built and evaluated an intent classification application using Orq. To further enhance your application:

- Explore other datasets and use cases.
- Integrate the model into a chatbot or voice assistant.
- Automate deployment and testing with Orq's advanced features.

For more resources, visit the Orq documentation.